<a href="https://colab.research.google.com/github/Mr-McCurdy/WhirlwindTourOfPython/blob/master/Canvas_NewQuiz_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 Canvas New‑Quiz ETL – Colab Template

**Paste any Canvas New‑Quiz URL** and run all cells to get two cleaned CSVs:

* `meta_clean.csv` – student‑level scores (+ pct column)  
* `items_clean.csv` – question‑level responses (if Canvas includes them)

> Requires a Canvas API token with `quizzes/reports` scope.


In [ ]:
!pip -q install pandas requests

## 1 Enter quiz URL and Canvas token

In [ ]:
from google.colab import userdata


# 🔑 EDIT THESE TWO VARIABLES
QUIZ_URL = input('Canvas New Quiz URL')
CANVAS_TOKEN = userdata.get('CANVAS_API_KEY')

REPORT_FMT = "json"   # 'json' or 'csv'

Canvas New Quiz URLhttps://pacificachristian.instructure.com/courses/2446/assignments/171369/edit?quiz_lti


## 2 Parse URL → domain / course / quiz

In [ ]:
import re, json, time, requests, pandas as pd
from urllib.parse import urlparse

m = re.search(r"/courses/(\d+)/assignments/(\d+)", QUIZ_URL)
if not m:
    raise ValueError("Could not parse COURSE/ASSIGNMENT IDs from URL.")
COURSE_ID, QUIZ_ID = m.groups()
DOMAIN = urlparse(QUIZ_URL).hostname
print(f"Domain={DOMAIN}  Course={COURSE_ID}  Quiz={QUIZ_ID}")

Domain=pacificachristian.instructure.com  Course=2446  Quiz=171369


## 3 Kick off report (student-analysis)

In [ ]:
HEADERS = {"Authorization": f"Bearer {CANVAS_TOKEN}"}
BASE = f"https://{DOMAIN}/api/quiz/v1/courses/{COURSE_ID}/quizzes/{QUIZ_ID}/reports"

payload = {"quiz_report[report_type]": "student_analysis",
           "quiz_report[format]": REPORT_FMT}

resp = requests.post(BASE, headers=HEADERS, data=payload).json()
progress_url = (resp.get("url") or resp.get("progress_url") or
                (resp.get("progress") or {}).get("url"))
if not progress_url:
    raise RuntimeError(json.dumps(resp, indent=2))
print("Progress URL:", progress_url)

Progress URL: https://pacificachristian.instructure.com/api/v1/progress/114048


## 4 Poll until completed

In [ ]:
while True:
    p = requests.get(progress_url, headers=HEADERS).json()
    if p["workflow_state"] == "completed":
        download_url = p["results"]["url"]; break
    if p["workflow_state"] == "failed":
        raise RuntimeError("Report failed")
    print("Waiting…", p.get("completion", "?"))
    time.sleep(3)

Waiting… None
Waiting… None
Waiting… None


## 5 Download & load raw DataFrame

In [ ]:
fname = f"raw_report.{REPORT_FMT}"
open(fname, "wb").write(requests.get(download_url, headers=HEADERS).content)

raw_df = pd.read_json(fname) if REPORT_FMT == "json" else pd.read_csv(fname)
print("Rows:", len(raw_df))
raw_df.head()

Rows: 176


,created_timestamp,student_data,item_responses,summary
0,2025-05-19T15:57:21.181Z,"{'id': 3474, 'name': 'Isla Raddon', 'uuid': 'x...","[{'item_id': '14725', 'answer': '10', 'item_ty...","{'number_of_correct': 11, 'number_of_incorrect..."
1,2025-05-19T15:57:21.189Z,"{'id': 3460, 'name': 'Wyatt Kemmerling', 'uuid...","[{'item_id': '14725', 'answer': '4', 'item_typ...","{'number_of_correct': 12, 'number_of_incorrect..."
2,2025-05-19T15:57:21.196Z,"{'id': 3465, 'name': 'Juliana Mitschiener', 'u...","[{'item_id': '14725', 'answer': '10', 'item_ty...","{'number_of_correct': 12, 'number_of_incorrect..."
3,2025-05-19T15:57:21.203Z,"{'id': 3728, 'name': 'Cole Arana', 'uuid': 'SI...","[{'item_id': '14725', 'answer': '10', 'item_ty...","{'number_of_correct': 19, 'number_of_incorrect..."
4,2025-05-19T15:57:21.209Z,"{'id': 3467, 'name': 'Maia Agadjanean', 'uuid'...","[{'item_id': '14725', 'answer': '10', 'item_ty...","{'number_of_correct': 12, 'number_of_incorrect..."


## 6 Helpers: flatten JSON columns

In [ ]:
import ast
from pandas import json_normalize

def _to_obj(x):
    if isinstance(x, (dict, list)): return x
    if pd.isna(x): return None
    try:
        return json.loads(x)
    except Exception:
        return ast.literal_eval(x)

def parse_newquiz_df(df,
                     student_col="student_data",
                     items_col="item_responses",
                     summary_col="summary"):

    df = df.copy()
    for c in (student_col, items_col, summary_col):
        if c in df.columns:
            df[c] = df[c].apply(_to_obj)

    meta = json_normalize(df[student_col]) if student_col in df else pd.DataFrame()
    summ = json_normalize(df[summary_col]) if summary_col in df else pd.DataFrame()
    meta_df = pd.concat([df.drop(columns=[c for c in (student_col, items_col, summary_col)
                                          if c in df], errors='ignore'),
                         meta.add_prefix("stu."),
                         summ.add_prefix("sum.")], axis=1)

    if items_col in df:
        items_df = (df[[student_col, items_col]]
                    .rename(columns={student_col:'_stu', items_col:'_items'})
                    .assign(student_id=lambda d:
                            d['_stu'].apply(lambda s: s.get("id") if isinstance(s, dict) else None))
                    .explode('_items').reset_index(drop=True))
        items_df = pd.concat([items_df[['student_id']],
                              json_normalize(items_df['_items'])], axis=1)
    else:
        items_df = pd.DataFrame()

    return meta_df, items_df

## 7 Flatten to `meta_df` / `items_df` and add `%`

In [ ]:
meta_df, items_df = parse_newquiz_df(raw_df)

if {'sum.score','sum.points_possible'}.issubset(meta_df.columns):
    meta_df['pct'] = meta_df['sum.score'] / meta_df['sum.points_possible']
elif {'sum.number_of_correct','sum.total_questions'}.issubset(meta_df.columns):
    meta_df['pct'] = meta_df['sum.number_of_correct'] / meta_df['sum.total_questions']

meta_df.head()

,created_timestamp,stu.id,stu.name,stu.uuid,stu.sis_id,stu.submitted_at,stu.elapsed_time,stu.attempt,stu.section_info.section_ids,stu.section_info.section_sis_ids,stu.section_info.section_names,sum.number_of_correct,sum.number_of_incorrect,sum.number_of_no_answer,sum.points_possible,sum.score,pct
0,2025-05-19T15:57:21.181Z,3474,Isla Raddon,xC3P4haWX2w1URUOJcN7Zkzdg6WsctVk4DDZY1Vp,usr-1054-8084337,2025-05-05T16:58:46.366Z,00:29:37,1,[3070],[],[All School Math Assessment],11,20,0,31.0,11.0,0.354839
1,2025-05-19T15:57:21.189Z,3460,Wyatt Kemmerling,b50gEY0d1zYfXjY9R5luxSSEWuR3i3YUr9jCDiAo,usr-1054-8081451,2025-05-05T17:01:52.508Z,00:32:47,1,[3070],[],[All School Math Assessment],12,19,0,31.0,12.0,0.387097
2,2025-05-19T15:57:21.196Z,3465,Juliana Mitschiener,M5akIr42UJfuuR8N6frClj8Wq1N4jAdnk2zuo65w,usr-1054-8081564,2025-05-05T17:03:05.023Z,00:37:25,1,[3070],[],[All School Math Assessment],12,19,0,31.0,12.0,0.387097
3,2025-05-19T15:57:21.203Z,3728,Cole Arana,SILcWX58cdYYK27FibKska0emfvB2MMsAp5N6Y1X,usr-1054-8352926,2025-05-05T17:21:30.352Z,00:25:57,1,[3070],[],[All School Math Assessment],19,12,0,31.0,19.0,0.612903
4,2025-05-19T15:57:21.209Z,3467,Maia Agadjanean,ilrtjYO5bHxBwuWdZUQq4PYYBQKg7MLn4i04qvIQ,usr-1054-7818827,2025-05-05T16:52:22.731Z,00:26:23,1,[3070],[],[All School Math Assessment],12,19,0,31.0,12.0,0.387097


## 8 Export cleaned CSVs

In [ ]:
meta_df.to_csv("meta_clean.csv", index=False)
if not items_df.empty:
    items_df.to_csv("items_clean.csv", index=False)
print("Saved meta_clean.csv", "& items_clean.csv" if not items_df.empty else "(no items file)")

Saved meta_clean.csv & items_clean.csv
